In [1]:
cd ..

/Users/dorislee/Desktop/Research/covid/covid19-vis


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/quarantine-activity-US-Apr16-long.csv")

In [4]:
import numpy as np

In [5]:
from chartlib.covid_chart import CovidChart
from chartlib.utils import create_lockdown_type, split_into_list, str2emo

In [6]:
# START modified from _ingest_usa_quarantine_df
quarantine_csv = "data/quarantine-activity-US-Apr16-long.csv"
quarantine_df = pd.read_csv(quarantine_csv)
groupcol = 'Province_State'

quarantine_df = quarantine_df.rename(columns={'State': 'Province_State', 'Effective Date': 'lockdown_date','Coverage.type':'Coverage'})
quarantine_df = quarantine_df.sort_values('Coverage', ascending=True)
quarantine_df['lockdown_type'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 0), axis=1)
quarantine_df['emoji_string'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 1), axis=1)
quarantine_df['lockdown_type'].replace('', np.nan, inplace=True)

quarantine_df = quarantine_df.dropna(subset=['lockdown_type'])
#TODO: this is where the columns get dropped
quarantine_df = quarantine_df.groupby(['lockdown_date', 'Province_State']).agg({
    'lockdown_type': lambda col: '; '.join(col),
    'emoji_string': lambda col: ''.join(col),
    'population_size': lambda col:col
}).reset_index()

quarantine_df.loc[quarantine_df.lockdown_type=="Regional Border Closure/Visitor Quarantine","emoji_string"]="t"#bugfix
# Breaking up emoji into separate rows for vertical stacking
quarantine_df.emoji_string = quarantine_df.emoji_string.apply(split_into_list)

quarantine_df = quarantine_df.explode(column='emoji_string')
quarantine_df['Coverage'] = quarantine_df.emoji_string.apply(
    lambda x: 'Statewide' if str(x).isupper() else 'Regional'
)
quarantine_df = quarantine_df.sort_values('Coverage', ascending=False)
quarantine_df.emoji_string = quarantine_df.emoji_string.str.lower()
quarantine_df['emoji'] = quarantine_df['emoji_string'].map(str2emo)
quarantine_df['event_index'] = quarantine_df.groupby(['Province_State', 'lockdown_date']).cumcount()

quarantine_cols = [
    groupcol, 'lockdown_date', 'lockdown_type', 'emoji', 'emoji_string', 'event_index', 'Coverage'
]
# quarantine_cols = ['Province_State', 'lockdown_date', 'lockdown_type', 'emoji']
quarantine_df = quarantine_df[quarantine_cols]

# END modified from _ingest_usa_quarantine_df

In [7]:
qdf = quarantine_df[quarantine_df["Province_State"]=="California"]

In [8]:
qdf

,Province_State,lockdown_date,lockdown_type,emoji,emoji_string,event_index,Coverage
120,California,3/19/20,Stay-at-home Order,🏠,l,0,Statewide
254,California,3/4/20,Declaration of Emergency,⚠️,e,0,Statewide
16,California,3/12/20,Gatherings (>250) Banned,👨‍👩‍👧‍👦,g,0,Statewide
31,California,3/13/20,Regional Closure of Schools; Regional Closure ...,🎓,s,0,Regional
31,California,3/13/20,Regional Closure of Schools; Regional Closure ...,🎓,s,1,Regional
31,California,3/13/20,Regional Closure of Schools; Regional Closure ...,🎓,s,2,Regional
31,California,3/13/20,Regional Closure of Schools; Regional Closure ...,🎓,s,3,Regional
31,California,3/13/20,Regional Closure of Schools; Regional Closure ...,🎓,s,4,Regional
31,California,3/13/20,Regional Closure of Schools; Regional Closure ...,🎓,s,5,Regional
31,California,3/13/20,Regional Closure of Schools; Regional Closure ...,🎓,s,6,Regional


In [24]:
quarantine_df_orig = pd.read_csv(quarantine_csv)

state_populations = quarantine_df_orig[quarantine_df_orig["Coverage.type"]=="State-wide"][["State","population_size"]].drop_duplicates()
# state_populations = state_populations.set_index(state_populations.State)
# state_populations = state_populations.drop(columns="State")
# state_populations_lookup= state_populations.population_size.to_dict()

In [25]:
state_populations = state_populations.rename(columns={"population_size":"state_population_size"})

In [26]:
#https://www.census.gov/quickfacts/PR
state_populations.loc[state_populations["State"]=="Puerto Rico","state_population_size"] = 3725789

In [27]:
state_populations.state_population_size = state_populations.state_population_size.astype("int")

In [28]:
quarantine_df_orig = quarantine_df_orig[['State', 'Coverage.type', 'Coverage.location', 'Effective Date','population_size']]
quarantine_df_orig = quarantine_df_orig.rename(columns={'State': 'Province_State', 'Effective Date': 'lockdown_date','Coverage.type':'Coverage'})

In [29]:
quarantine_df = quarantine_df.merge(quarantine_df_orig)

In [30]:
quarantine_df = quarantine_df.merge(state_populations,right_on="State",left_on="Province_State")

In [31]:
severityScore = {'g':0.1, 's':0.5, 'r':0.4, 'n':0.7, 'l':1, 't':0.3, 'c':0.4, 'e':0.2}

In [32]:
# quarantine_df.apply(lambda x: int(x["population_size"])/int(state_populations_lookup[x["Province_State"]]),axis=1)

In [41]:
# quarantine_df[quarantine_df["population_size"].isna()]
quarantine_df = quarantine_df.dropna(subset=["population_size"])

In [47]:
quarantine_df.population_size = quarantine_df.population_size.str.replace(",","")

In [49]:
quarantine_df.population_size = quarantine_df.population_size.astype("int")

In [51]:
quarantine_df["pctStateAffected"]= quarantine_df["population_size"]/quarantine_df["state_population_size"]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
quarantine_df[quarantine_df["pctStateAffected"]==1]

,Province_State,lockdown_date,lockdown_type,emoji,emoji_string,event_index,Coverage,Coverage.location,population_size,State_x,state_population_size_x,State_y,state_population_size_y,State,state_population_size,pctStateAffected
